<a href="https://colab.research.google.com/github/yorkjong/stock-reports/blob/main/notebooks/ibd_reports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install and Setup (this section will be executed automatically)

#### Install Required Packages

In [1]:
%pip install "git+https://github.com/yorkjong/vistock.git@feature/ibd"
%pip install requests-cache

  Cloning https://github.com/yorkjong/vistock.git (to revision feature/ibd) to /tmp/pip-req-build-3oi93g7w
  Running command git clone --filter=blob:none --quiet https://github.com/yorkjong/vistock.git /tmp/pip-req-build-3oi93g7w
  Running command git checkout -b feature/ibd --track origin/feature/ibd
  Switched to a new branch 'feature/ibd'
  Branch 'feature/ibd' set up to track remote branch 'feature/ibd' from 'origin'.
  Resolved https://github.com/yorkjong/vistock.git to commit c92db175b3aaa067e269a55e20e55ab71f9f8ea4
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.1 MB/s eta 0:00:00
  Created wheel for vistock: filename=vistock-0.5.0-py3-none-any.whl size=72935 sha256=c4c8758be3ee87b931c1be317f9cbac2eefa74671df30a28efa46e1275e3e871
  Stored in directory: /tmp/pip-ephem-wheel-cache-dn7tpams/wheels/07/97/7f/a72c7fbc975be0326ac45fec1ca0c872e2d6e2bbea96caf706
Successfully built vistock
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4

In [2]:
# @title Install Cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

#### Setup and Configuration

In [3]:
# @title Checkboxes

import ipywidgets as widgets

def cbs_create(symbols, n_pre_checked=10):
    '''Create a list of checkboxes'''
    return [
        widgets.Checkbox(
            value=(i < n_pre_checked),  # Set first n items as checked
            description=symbol,
            layout=widgets.Layout(width='auto'),
            style={'description_width': 'auto'}
        )
        for i, symbol in enumerate(symbols)
    ]

def cbs_with_grid(checkboxes, n_cols=5):
    '''Create a grid layouting the given checkboxes'''
    return widgets.GridBox(checkboxes, layout=widgets.Layout(
        width='auto',
        grid_template_columns=f'repeat({n_cols}, 1fr)',
        grid_gap='10px'  # Add some space between the checkboxes
    ))

def cbs_get_selected(checkboxes):
    '''Get the selected symbols from the given checkboxes'''
    return [checkbox.description for checkbox in checkboxes if checkbox.value]

In [4]:
# @title Enable Requests Cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

In [5]:
# @title ReadOnlyGitHub
import requests
import pandas as pd
from io import StringIO

class ReadOnlyGitHub:
    def __init__(self, repo_owner, repo_name, dir='', branch='main'):
        dir = dir.strip('/')
        base = 'https://raw.githubusercontent.com'
        if dir:
            self.raw_url = f'{base}/{repo_owner}/{repo_name}/{branch}/{dir}'
        else:
            self.raw_url = f'{base}/{repo_owner}/{repo_name}/{branch}'

        base = 'https://api.github.com/repos'
        if dir:
            self.api_url = f'{base}/{repo_owner}/{repo_name}/contents/{dir}'
        else:
            self.api_url = f'{base}/{repo_owner}/{repo_name}/contents'

    def file_exists(self, file_path):
        url = f'{self.raw_url}/{file_path}'
        response = requests.head(url)
        return response.status_code == 200

    def list_filenames(self, dir_path=''):
        url = f'{self.api_url}/{dir_path}'
        response = requests.get(url)
        if response.status_code == 200:
            files = response.json()
            return [item['name'] for item in files]
        else:
            print(f"Request failed: {response.status_code} - {response.text}")
            return []

    def download_csv(self, file_path):
        url = f'{self.raw_url}/{file_path}'
        if self.file_exists(file_path):
            return pd.read_csv(url)
        else:
            return pd.DataFrame()

# Create a GitHub instance
github = ReadOnlyGitHub(
    repo_owner='YorkJong',
    repo_name='stock-reports',
    dir='data/ibd'
)

In [6]:
# @title DataFrame Operations
import yfinance as yf
import vistock.tw as tw

def filter_increasing_relative_strength(df):
    """
    Filter stocks with increasing Relative Strength over different time periods.

    This function filters the DataFrame to include only those stocks where:
    - Relative Strength is above 100.
    - Relative Strength has increased over the past 1 month, 3 months, and 6 months.
    Optionally, you can add a condition to check if Percentile is above 90.
    """
    return df[
        (df["Relative Strength"] > 100)
        & (df["Relative Strength"] > df["1 Month Ago"])
        & (df["1 Month Ago"] > df["3 Months Ago"])
        & (df["3 Months Ago"] > df["6 Months Ago"])
        # & (df["Percentile"] > 90)  # Uncomment to include Percentile filter
    ]

def df_tickers_str_to_names_str(df):
    if 'Tickers' in df.columns:
        df['Tickers'] = df['Tickers'].apply(
            lambda x: ','.join([tw.stock_name(ticker)
            for ticker in x.split(',')])
        )

def df_tickers_str_to_names(df):
    if 'Tickers' in df.columns:
        #df['Tickers'] = df['Tickers'].str.split(',').apply(
        #    lambda x: [tw.stock_name(ticker) for ticker in x])
        df['Tickers'] = df['Tickers'].apply(
            lambda x: [tw.stock_name(ticker) for ticker in x.split(',')]
        )

In [7]:
# @title OrderedSet

class OrderedSet:
    def __init__(self, iterable=[]):
        self._data = dict.fromkeys(iterable)

    def add(self, item):
        self._data[item] = None

    def discard(self, item):
        self._data.pop(item, None)

    def __contains__(self, item):
        return item in self._data

    def __iter__(self):
        return iter(self._data)

    def __len__(self):
        return len(self._data)

    def __repr__(self):
        return f"OrderedSet({list(self._data)})"

    def union(self, other):
        """Return a new OrderedSet with elements from both sets."""
        combined = OrderedSet(self._data.keys())
        combined._data.update(other._data)
        return combined

    def update(self, other):
        """Update the current OrderedSet with elements from another OrderedSet."""
        self._data.update(other._data)

    def __add__(self, other):
        """Concatenate two OrderedSets."""
        combined = OrderedSet(self._data.keys())
        combined.update(other)
        return combined

In [8]:
# @title enable_plotly_in_cell
# ref. https://stackoverflow.com/questions/76593068/plotly-figure-not-rendering-in-ipywidgets-interact-function-google-colab
def enable_plotly_in_cell():
  import IPython
  from plotly.offline import init_notebook_mode
  display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
  init_notebook_mode(connected=False)

### Glossary of Terms

source (The source of stocks to analyze):
- This could include stocks traded on exchanges or components of a specific index.
- Common abbreviation(s) for the exchange or market sector.  
  - For Taiwan Markets, possible values include:
    - `TWSE`: Taiwan Stock Exchange (台灣上市股票交易所）
    - `TPEX`: Taipei Exchange （上櫃交易所）
    - `ESB`: Emerging Stock Board （興櫃交易所）
  - Can also be combined with '+' (e.g., `TWSE+TPEX`, `TWSE+TPEX+ESB`)
  - For America Markets, possible values include:
    - `SPX`: S&P 500 (標普五百指數)
    - `DJIA`: Dow Jones Industrial Average (道瓊指數)
    - `NDX`: NASDAQ-100 (納斯達克一百指數)
    - `SOX`: PHLX Semiconductor Index （費半指數）
  - Multiple indices can be combined using '+' (e.g., `SPX+DJIA+NDX+SOX`)

period (Historical Data Time Range)：
- The time range for which to fetch historical data.
- `2y` means 2 years
- `6mo` means 6 monthes

RS (Relative Strength)
- Relative Strength (RS) is a metric used to evaluate the performance of a stock relative to a benchmark index.
  - A higher RS rating indicates that the stock has outperformed the index, while a lower RS rating suggests underperformance.
- The IBD RS calculates the performance of the last year, with the most recent quarter weighted double.


### Execute Actions Step by Step


In [9]:
# @title Step 1. Pick a File

import ipywidgets as widgets

with requests_cache.disabled():
    filenames = github.list_filenames()

# Extract and sort all unique values in reverse order
all_dates = sorted(set(fn.split('_')[3].replace('.csv', '') for fn in filenames), reverse=True)
all_sources = sorted(set(fn.split('_')[0] for fn in filenames), reverse=True)
all_types = sorted(set(fn.split('_')[1] for fn in filenames), reverse=True)
all_periods = sorted(set(fn.split('_')[2] for fn in filenames), reverse=True)

# Calculate the maximum length of filenames to set dropdown width
max_filename_length = max(len(fn) for fn in filenames)
dropdown_width = f'{max_filename_length * 10}px'  # 10px width per character

# Create dropdowns with dynamic width
date_dropdown = widgets.Dropdown(
    options=all_dates,
    description='Date:',
    layout=widgets.Layout(width=dropdown_width)
)

source_dropdown = widgets.Dropdown(
    options=all_sources,
    description='Source:',
    layout=widgets.Layout(width=dropdown_width)
)

type_dropdown = widgets.Dropdown(
    options=all_types,
    description='Type:',
    layout=widgets.Layout(width=dropdown_width)
)

period_dropdown = widgets.Dropdown(
    options=all_periods,
    description='Period:',
    layout=widgets.Layout(width=dropdown_width)
)

# Create file selection dropdown
file_dropdown = widgets.Dropdown(
    options=[],
    description='File:',
    layout=widgets.Layout(width=dropdown_width)
)

def update_dropdowns(*args):
    # Filter files based on selected date
    date_filtered_files = [fn for fn in filenames if date_dropdown.value in fn]

    # Update Source dropdown
    available_sources = sorted(set(fn.split('_')[0] for fn in date_filtered_files), reverse=True)
    source_dropdown.options = available_sources
    if source_dropdown.value not in available_sources:
        source_dropdown.value = available_sources[0] if available_sources else None

    # Update Type dropdown
    available_types = sorted(set(fn.split('_')[1] for fn in date_filtered_files), reverse=True)
    type_dropdown.options = available_types
    if type_dropdown.value not in available_types:
        type_dropdown.value = available_types[0] if available_types else None

    # Update Period dropdown
    available_periods = sorted(set(fn.split('_')[2] for fn in date_filtered_files), reverse=True)
    period_dropdown.options = available_periods
    if period_dropdown.value not in available_periods:
        period_dropdown.value = available_periods[0] if available_periods else None

    # Update file options
    update_file_options()

def update_file_options(*args):
    filtered_files = [
        fn for fn in filenames
        if (date_dropdown.value in fn and
            source_dropdown.value == fn.split('_')[0] and
            type_dropdown.value == fn.split('_')[1] and
            period_dropdown.value == fn.split('_')[2])
    ]
    file_dropdown.options = filtered_files
    if filtered_files:
        file_dropdown.value = filtered_files[0]  # Set initial value to the first match
    else:
        file_dropdown.value = None

# Bind event handlers
date_dropdown.observe(update_dropdowns, 'value')
source_dropdown.observe(update_file_options, 'value')
type_dropdown.observe(update_file_options, 'value')
period_dropdown.observe(update_file_options, 'value')

# Display all dropdowns
display(date_dropdown, source_dropdown, type_dropdown, period_dropdown, file_dropdown)

# Initialize dropdowns
update_dropdowns()

Dropdown(description='Date:', layout=Layout(width='470px'), options=('20240827',), value='20240827')

Dropdown(description='Source:', layout=Layout(width='470px'), options=('TWSE+TPEX', 'TWSE', 'TPEX', 'SPX+DJIA+…

Dropdown(description='Type:', layout=Layout(width='470px'), options=('stocks', 'industries'), value='stocks')

Dropdown(description='Period:', layout=Layout(width='470px'), options=('2y',), value='2y')

Dropdown(description='File:', layout=Layout(width='470px'), options=(), value=None)

In [13]:
# @title Step 2. Print Top Percentile Items {"run":"auto"}
num_items = 10 # @param [10, 20, 30] {"type":"raw"}

selected_file = file_dropdown.value
source, kind, period, date = selected_file.split('_')
date = date.replace('.csv', '')
print(f"Source: {source}\nType: {kind}\nPeriod: {period}\nDate: {date}")

df = github.download_csv(selected_file)
df_top_s1 = df.head(num_items).copy()
df_tickers_str_to_names_str(df_top_s1)
display(df_top_s1)

if 'Name' in df_top_s1.columns:
    print(list(df_top_s1['Name']))
if 'Ticker' in df_top_s1.columns:
    print(list(df_top_s1['Ticker']))

Source: TWSE+TPEX
Type: stocks
Period: 2y
Date: 20240827


,Ticker,Name,Price,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
0,6144,得利影,98.50,Communication Services,Entertainment,502.53,454.28,149.16,92.40,99,99,95,47,1
1,6640,均華,999.00,Technology,Semiconductors,424.46,416.15,237.35,236.22,99,99,99,99,2
2,1799,易威,218.00,Healthcare,Drug Manufacturers—Specialty & Generic,419.97,325.99,258.08,202.10,99,99,99,98,3
3,6442,光聖,448.50,Technology,Electronic Components,374.24,409.51,183.52,213.00,99,99,98,98,4
4,8374,羅昇,138.00,Industrials,Industrial Distribution,334.70,334.99,123.40,82.34,99,99,90,15,5
5,4513,福裕,78.90,Industrials,Specialty Industrial Machinery,299.55,178.23,152.50,96.35,99,97,96,56,6
6,3535,晶彩科,80.30,Technology,Computer Hardware,298.84,290.85,122.94,107.60,99,99,90,75,7
7,3230,錦明,43.15,Technology,Electronic Components,298.63,173.51,96.36,86.19,99,97,69,27,8
8,4562,穎漢,73.00,Industrials,Specialty Industrial Machinery,295.22,292.71,173.37,88.59,99,99,97,35,9
9,5484,慧友,82.80,Industrials,Security & Protection Services,288.25,235.02,87.96,123.93,99,98,54,86,10


['得利影', '均華', '易威', '光聖', '羅昇', '福裕', '晶彩科', '錦明', '穎漢', '慧友']
['6144', '6640', '1799', '6442', '8374', '4513', '3535', '3230', '4562', '5484']


In [11]:
# @title Step 3. Filter Items with Increasing RS > 100 {"run":"auto"}
num_items = 10 # @param [10, 20, 30] {"type":"raw"}

selected_file = file_dropdown.value
source, kind, period, date = selected_file.split('_')
date = date.replace('.csv', '')
print(f"Source: {source}\nType: {kind}\nPeriod: {period}\nDate: {date}")

df = github.download_csv(selected_file)
filtered_df = filter_increasing_relative_strength(df)
df_top_s2 = filtered_df.head(num_items).copy()
df_tickers_str_to_names_str(df_top_s2)
display(df_top_s2)

if 'Name' in df_top_s2.columns:
    print(list(df_top_s2['Name']))
if 'Ticker' in df_top_s2.columns:
    print(list(df_top_s2['Ticker']))

Source: TWSE+TPEX
Type: stocks
Period: 2y
Date: 20240827


,Ticker,Name,Price,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
0,6144,得利影,98.50,Communication Services,Entertainment,502.53,454.28,149.16,92.40,99,99,95,47,1
1,6640,均華,999.00,Technology,Semiconductors,424.46,416.15,237.35,236.22,99,99,99,99,2
2,1799,易威,218.00,Healthcare,Drug Manufacturers—Specialty & Generic,419.97,325.99,258.08,202.10,99,99,99,98,3
5,4513,福裕,78.90,Industrials,Specialty Industrial Machinery,299.55,178.23,152.50,96.35,99,97,96,56,6
6,3535,晶彩科,80.30,Technology,Computer Hardware,298.84,290.85,122.94,107.60,99,99,90,75,7
7,3230,錦明,43.15,Technology,Electronic Components,298.63,173.51,96.36,86.19,99,97,69,27,8
8,4562,穎漢,73.00,Industrials,Specialty Industrial Machinery,295.22,292.71,173.37,88.59,99,99,97,35,9
11,2524,京城,118.00,Real Estate,Real Estate—Development,277.22,188.66,119.67,88.28,99,97,88,34,12
15,8096,擎亞,56.40,Technology,Semiconductors,268.62,202.73,202.49,199.50,99,98,98,98,16
25,3284,太普高,57.90,Industrials,Specialty Industrial Machinery,233.61,172.34,144.86,118.91,98,97,95,83,26


['得利影', '均華', '易威', '福裕', '晶彩科', '錦明', '穎漢', '京城', '擎亞', '太普高']
['6144', '6640', '1799', '4513', '3535', '3230', '4562', '2524', '8096', '3284']


In [ ]:
# @title Step 4. Show the Comparison of RS Lines {"run":"auto"}
source = "Major Global Stock Indices" # @param ["Step 2. Top Percentile Items","Step 3. Top Increasing RS","Major Global Stock Indices"]
period = "2y" # @param ["1y","2y","5y"]
interval = "1d" # @param ["1d","1wk"]
backend = "mplfinance" # @param ["mplfinance","Plotly"]

import ipywidgets as widgets
from IPython.display import display, clear_output

MAX_STOCK_SELECTION = 10
N_COLS = 8  # the number of columns of the grid layout for checkboxes

def get_tickers(df):
    return [ticker for sublist in df['Tickers'] for ticker in sublist]

def convert_to_list(value):
    if isinstance(value, str):
        return value.split(',')
    return value

def tickers_from_df(df):
    tickers = OrderedSet()
    if 'Ticker' in df.columns:
        tickers += OrderedSet(df['Ticker'])
    elif 'Tickers' in df.columns:
        df['Tickers'] = df['Tickers'].apply(convert_to_list)
        tickers += OrderedSet(get_tickers(df))
    return tickers

def top_tickers():
    return tickers_from_df(df_top_s1)

def top_filtered_tickers():
    return tickers_from_df(df_top_s2)

def major_indices():
    return ['^DJI', '^IXIC', '^NDX', '^RUT', '^SOX',
            '^TWII', '^N225', '^HSI',
            '^STOXX50E', '^FTSE', '^GDAXI', '^FCHI', '^GSPTSE']

tickers = {
    'Step 2. Top Percentile Items': top_tickers,
    'Step 3. Top Increasing RS': top_filtered_tickers,
    'Major Global Stock Indices': major_indices,
}[source]()
symbols = [tw.stock_name(t) for t in tickers]

checkboxes = cbs_create(symbols, MAX_STOCK_SELECTION)
checkbox_grid = cbs_with_grid(checkboxes, N_COLS)
button = widgets.Button(description="Generate Plot")
out_msg = widgets.Output()
out_fig = widgets.Output()

style_dropdown = widgets.Dropdown(
    options=['default', 'classic', 'yahoo', 'charles', 'tradingview', 'binance', 'binancedark', 'mike', 'nightclouds', 'checkers', 'ibd', 'sas', 'starsandstripes', 'kenan', 'blueskies', 'brasil'],
    value='checkers',
    description='Style:',
)
template_dropdown = widgets.Dropdown(
    options=['plotly', 'plotly_white', 'plotly_dark', 'ggplot2', 'seaborn', 'simple_white', 'presentation', 'xgridoff', 'ygridoff'],
    value='plotly_dark',
    description='Template:',
)

# Dropdown menu for style/template
if backend == 'mplfinance':
    display(checkbox_grid, style_dropdown, button, out_msg, out_fig)
else: # Plotly
    display(checkbox_grid, template_dropdown, button, out_msg, out_fig)

import vistock.mpl as vs_mpl
import vistock.plotly as vs_plotly

ibd_rs_cmp = {
    'mplfinance': vs_mpl.ibd_rs_cmp,
    'Plotly': vs_plotly.ibd_rs_cmp,
}[backend]

def on_checkbox_change(change):
    selected_count = sum([cb.value for cb in checkboxes])
    if selected_count > MAX_STOCK_SELECTION:
        # Uncheck the last checked box if selection exceeds limit
        changed_checkbox = change['owner']
        changed_checkbox.value = False
        with out_msg:
            out_fig.clear_output()
            print(f"Only {MAX_STOCK_SELECTION} stocks can be selected at most.")

# Bind the checkbox change event to the function
for checkbox in checkboxes:
    checkbox.observe(on_checkbox_change, names='value')

def on_button_click(b):
    symbols = cbs_get_selected(checkboxes)
    if not symbols:
        with out_msg:
            out_fig.clear_output()
            print("No stocks selected. Please select at least one stock.")
        return
    with out_fig:
        out_msg.clear_output()
        clear_output()
        if backend == 'mplfinance':
            ibd_rs_cmp.plot(symbols, interval=interval, period=period,
                            style=style_dropdown.value)
        else: # Plotly
            ibd_rs_cmp.plot(symbols, interval=interval, period=period,
                            template=template_dropdown.value)

button.on_click(on_button_click)

if backend == 'Plotly':
    enable_plotly_in_cell()


GridBox(children=(Checkbox(value=True, description='^DJI', layout=Layout(width='auto'), style=DescriptionStyle…

Dropdown(description='Style:', index=9, options=('default', 'classic', 'yahoo', 'charles', 'tradingview', 'bin…

Button(description='Generate Plot', style=ButtonStyle())

Output()

Output()